*************************************************************************************************************************
#  Uses Carver legacy files to reconstruct raw data
## Description...
Historic data is not available from IB. The 'cheapest' source is Quandl. However some maturites for some contracts have bad/missing data. Use legacycsv files (PRICE and CARRY) to reconstruct the raw maturities and use these where Quandl data is lacking

### legacycsv fiels

The legacycsv CARRY files contain all the raw data. However on each roll date although the 'new' maturity raw data price is available, the 'old' price is not. This notebook reconstructs the roll date 'old' price so that the successive maturites can be used to reconstruct each 'panama stitched' PRICE series in combination with raw files from Quandl (or IB)

### Usage
get_legacy_raw(contract, path)
function
parameters: contract: symbol
            legacypath: path to legacy files
returns: raw price series (DATETIME, PRICE_CONTRACT, PRICE)

*************************************************************************************************************************

In [3]:
# Imports
import numpy as np
import pandas as pd
legacydir = "legacycsv/"

In [9]:
# Reconstructing raw data for PRICE maturities (NB Only PRICE series is used for stitching...)
# Firstjoin CARRY and PRICE files....
contract = "AEX"
filec = legacydir + contract + "_carrydata.csv"
filep = legacydir  + contract + "_price.csv"
df_carry = pd.read_csv(filec, index_col='DATETIME', dtype={'CARRY_CONTRACT': str, 'PRICE_CONTRACT': str})
df_price = pd.read_csv(filep, index_col='DATETIME')
df_price.columns = ['PANPRICE']
df_result = pd.concat([df_carry, df_price], axis=1, join='outer')

# From df_result, select only the rows on roll date and day before roll date
df_first = df_result.reset_index().groupby('PRICE_CONTRACT').first()
df_last = df_result.reset_index().groupby('PRICE_CONTRACT').last()
df_firstlast = pd.concat([df_first, df_last])
df_firstlast.reset_index(inplace=True)
df_firstlast.set_index('DATETIME', inplace=True)
df_firstlast.sort_index(inplace=True)
df_firstlast['DELTA'] = df_firstlast.PANPRICE - df_firstlast.PRICE   # Delta is difference between raw and stitched prices
df_firstlast.head()

,PRICE_CONTRACT,PRICE,CARRY,CARRY_CONTRACT,PANPRICE,DELTA
DATETIME,,,,,,
2013-07-26,201310,370.55,368.25,201311,344.166346,-26.383654
2013-10-17,201310,383.60,382.50,201311,357.216346,-26.383654
2013-10-18,201311,384.80,385.00,201312,359.516346,-25.283654
2013-11-14,201311,395.50,395.70,201312,370.216346,-25.283654
2013-11-15,201312,396.15,395.00,201401,370.662500,-25.487500


In [10]:
# Expired maturity PRICE on roll date is reconstructed from the Current Maturity Raw PRICE and 
# change in DELTA on RollDate and on RollDate[-1]. Shifting PRICE_CONTRACT and DELTA columns 
# places the previous row's values on the current row. Hence...
df_firstlast['MATSHIFT'] = df_firstlast.PRICE_CONTRACT.shift(1)
df_firstlast['DELTASHIFT'] = df_firstlast.DELTA.shift(1)
# Perform calculation to reconstruct PRICE of expired maturity on roll date
df_firstlast['REC_PRICE'] = df_firstlast.PRICE - (df_firstlast.DELTA - df_firstlast.DELTASHIFT)
df_firstlast.head()
df_firstlast.tail()

,PRICE_CONTRACT,PRICE,CARRY,CARRY_CONTRACT,PANPRICE,DELTA,MATSHIFT,DELTASHIFT,REC_PRICE
DATETIME,,,,,,,,,
2013-07-26,201310,370.55,368.25,201311,344.166346,-26.383654,NaN,NaN,NaN
2013-10-17,201310,383.60,382.50,201311,357.216346,-26.383654,201310,-26.383654,383.600000
2013-10-18,201311,384.80,385.00,201312,359.516346,-25.283654,201310,-26.383654,383.700000
2013-11-14,201311,395.50,395.70,201312,370.216346,-25.283654,201311,-25.283654,395.500000
2013-11-15,201312,396.15,395.00,201401,370.662500,-25.487500,201311,-25.283654,396.353846


,PRICE_CONTRACT,PRICE,CARRY,CARRY_CONTRACT,PANPRICE,DELTA,MATSHIFT,DELTASHIFT,REC_PRICE
DATETIME,,,,,,,,,
2016-09-09,201609,448.300,448.35,201610,445.500,-2.80,201609,-2.80,448.300
2016-09-12,201610,449.900,449.90,201611,449.050,-0.85,201609,-2.80,447.950
2016-10-14,201610,450.300,448.80,201611,449.450,-0.85,201610,-0.85,450.300
2016-10-17,201611,446.650,446.65,201612,446.650,0.00,201610,-0.85,445.800
2016-11-11,201611,445.725,452.10,201612,445.725,0.00,201611,0.00,445.725


In [29]:
# Retain only the rolldate rows
df_recon = df_firstlast.reset_index().groupby('PRICE_CONTRACT').first()
df_recon.tail()
# These rows can now be added to the PRICE rows and individual matuties can be used where Quandl history is missing to 
# initialize Panama series
df_recon.reset_index(inplace=True)
df_recon.set_index('DATETIME', inplace=True)
dframe = df_recon[['MATSHIFT','REC_PRICE']]
dframe.columns = ['PRICE_CONTRACT','PRICE']
dframe.head()

,DATETIME,PRICE,CARRY,CARRY_CONTRACT,PANPRICE,DELTA,MATSHIFT,DELTASHIFT,REC_PRICE
PRICE_CONTRACT,,,,,,,,,
201607,2016-06-13,425.00,425.00,201608,417.85,-7.15,201606,-8.30,423.85
201608,2016-07-13,442.05,440.85,201609,438.40,-3.65,201607,-7.15,438.55
201609,2016-08-18,450.35,449.65,201610,447.55,-2.80,201608,-3.65,449.50
201610,2016-09-12,449.90,449.90,201611,449.05,-0.85,201609,-2.80,447.95
201611,2016-10-17,446.65,446.65,201612,446.65,0.00,201610,-0.85,445.80


,PRICE_CONTRACT,PRICE
DATETIME,,
2013-07-26,201310,383.600000
2013-10-18,201310,383.700000
2013-11-15,201311,396.353846
2013-12-18,201312,381.381250
2014-01-17,201401,406.356250


In [31]:
df_raw = pd.concat([df_carry[['PRICE_CONTRACT','PRICE']], dframe])
df_raw.sort_index(inplace=True)
df_raw.head()

,PRICE_CONTRACT,PRICE
DATETIME,,
2013-07-26,201310,370.55
2013-07-26,201310,383.60
2013-07-29,NaN,NaN
2013-07-30,NaN,NaN
2013-07-31,NaN,NaN


In [32]:
df_raw[df_raw['PRICE_CONTRACT'] == '201310'].head()

,PRICE_CONTRACT,PRICE
DATETIME,,
2013-07-26,201310,370.55
2013-07-26,201310,383.60
2013-08-15,201310,375.35
2013-08-16,201310,374.25
2013-08-19,201310,373.85
